In [1]:
import wandb
import yaml
from utils import correct_type_of_entry
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import product
pd.set_option('display.max_rows', None)

/Users/mathieubazinet/.pyenv/versions/3.12.3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = "airfoil"
sweep_config = "tree_based"
values_to_fetch = ['kl_bound', 'compression_set_size', 'validation_loss', 'test_loss']

In [3]:
sweep_config_name = "./configs/sweep_configs/" + sweep_config + ".yaml"
with open(sweep_config_name) as file:
    sweep_configuration = yaml.safe_load(file)
    
hps = {}
for key, item in sweep_configuration['parameters'].items():
    if item.get('values', None) is not None:
        hps[key] = correct_type_of_entry(item['values'])
size_hyperparams = tuple([len(l) for l in hps.values()])

In [4]:
api = wandb.Api()
entity = "mathieu-bazinet"
project = "sweep" + dataset
runs = api.runs(entity + "/" + project)

In [5]:
results_matrix = np.ones(((len(values_to_fetch),) + size_hyperparams))

for run_idx in range(len(runs)):
    run = runs[run_idx]
    for val_to_fetch_idx in range(len(values_to_fetch)):
        matrix_idx = tuple([val_to_fetch_idx] + [hps[key].index(run.config[key]) for key in hps.keys()])
        val_to_fetch = values_to_fetch[val_to_fetch_idx]
        try:
            results_matrix[matrix_idx] = run.summary[val_to_fetch]
        except KeyError:
            results_matrix[matrix_idx] = [row[val_to_fetch] for row in run.scan_history(keys=[val_to_fetch])][-1]

ValueError: 0.2 is not in list

In [45]:
hp_list = list(hps.values())[1:]
params_product = list(product(*hp_list))
name_list = []
idx_list = []
for params in params_product:
    name = ""
    for p in params:
        name += str(p) + " "
    name_list.append(name[:-1])
    idx = ()
    for p_idx in range(len(params)):
        p_key = list(hps.keys())[1:][p_idx]
        idx += (hps[p_key].index(params[p_idx]),)
    idx_list.append(tuple(idx))

In [48]:
reshaped_matrix = results_matrix.mean(1).reshape(results_matrix.shape[0],np.prod(results_matrix.shape[2:])).T
mean_df = pd.DataFrame(reshaped_matrix, index=name_list, columns=values_to_fetch)
mean_df

,kl_bound,compression_set_size,validation_loss,test_loss
tree 5 2 1 50 0.0,24.692004,46.0,11.217294,11.259048
tree 5 2 1 50 0.01,24.856425,46.0,11.217436,11.177604
tree 5 2 1 50 0.05,24.856425,46.0,11.217436,11.177604
tree 5 2 1 100 0.0,24.856425,46.0,11.217436,11.177604
tree 5 2 1 100 0.01,24.856425,46.0,11.217436,11.177604
tree 5 2 1 100 0.05,24.856425,46.0,11.217436,11.177604
tree 5 2 2 50 0.0,18.998390,12.2,10.890239,11.098309
tree 5 2 2 50 0.01,18.998390,12.2,10.890239,11.098309
tree 5 2 2 50 0.05,18.998390,12.2,10.890239,11.098309
tree 5 2 2 100 0.0,18.998390,12.2,10.890239,11.098309
